In [3]:
import pandas as pd
import torch
import numpy as np
from torch import nn, optim
import torch.utils.data as data

In [4]:
device = torch.device('cuda')

In [13]:
torch.cuda.manual_seed_all(2137)
torch.manual_seed(2137)

In [14]:
df = pd.read_csv('train_data.csv')
test_df = pd.read_csv('test_data.csv')

In [ ]:
df

In [ ]:
test_df

problematyczne kolumny:

MonthSold - rozbić na klasy lub wywalić

HallwayType, HeatingType, AptManageType, SubwayStation - rozbić na klasy

TimeToBusStop, TimeToSubway - rozbić na klasy albo przerobić na liczby

In [15]:
categorical_columns = ['HallwayType', 'MonthSold', 'HeatingType', 'AptManageType', 'SubwayStation', 'TimeToBusStop', 'TimeToSubway']
df = df.astype({'MonthSold': str})
categorical_values = pd.get_dummies(df[categorical_columns])
categorical_values.head()

,HallwayType_corridor,HallwayType_mixed,HallwayType_terraced,MonthSold_1,MonthSold_10,MonthSold_11,MonthSold_12,MonthSold_2,MonthSold_3,MonthSold_4,...,SubwayStation_Sin-nam,SubwayStation_no_subway_nearby,TimeToBusStop_0~5min,TimeToBusStop_10min~15min,TimeToBusStop_5min~10min,TimeToSubway_0-5min,TimeToSubway_10min~15min,TimeToSubway_15min~20min,TimeToSubway_5min~10min,TimeToSubway_no_bus_stop_nearby
0,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
2,0,0,1,0,0,0,0,0,0,0,...,1,0,1,0,0,1,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0


In [16]:
numeric_values = df.drop(columns=categorical_columns)
numeric_values['SalePrice'] = numeric_values['SalePrice'].apply(lambda b: 1 if b > 300000 else 0)
train_indices = np.random.rand(len(numeric_values))>0.3

In [24]:
numeric_values

,SalePrice,YearBuilt,YrSold,Size(sqf),Floor,N_Parkinglot(Ground),N_Parkinglot(Basement),N_APT,N_manager,N_elevators,...,N_FacilitiesNearBy(Mall),N_FacilitiesNearBy(ETC),N_FacilitiesNearBy(Park),N_SchoolNearBy(Elementary),N_SchoolNearBy(Middle),N_SchoolNearBy(High),N_SchoolNearBy(University),N_FacilitiesInApt,N_FacilitiesNearBy(Total),N_SchoolNearBy(Total)
0,0,1985,2007,587,8,80.0,76.0,1.0,2.0,2.0,...,1.0,2.0,1.0,2.0,1.0,1.0,0.0,3,12.0,4.0
1,0,1985,2007,587,6,80.0,76.0,1.0,2.0,2.0,...,1.0,2.0,1.0,2.0,1.0,1.0,0.0,3,12.0,4.0
2,1,2006,2007,2056,8,249.0,536.0,6.0,5.0,11.0,...,1.0,0.0,0.0,2.0,2.0,1.0,2.0,5,3.0,7.0
3,0,1993,2007,1761,3,523.0,536.0,8.0,8.0,20.0,...,1.0,5.0,0.0,4.0,3.0,5.0,5.0,4,14.0,17.0
4,0,1992,2007,355,5,200.0,0.0,3.0,5.0,10.0,...,1.0,5.0,1.0,4.0,3.0,5.0,5.0,3,16.0,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4119,1,2007,2017,1643,19,0.0,1270.0,7.0,14.0,16.0,...,1.0,0.0,2.0,3.0,3.0,2.0,2.0,10,9.0,10.0
4120,0,2006,2017,903,13,123.0,181.0,3.0,3.0,11.0,...,1.0,2.0,0.0,4.0,3.0,3.0,1.0,4,8.0,11.0
4121,1,2007,2017,868,20,0.0,1270.0,7.0,14.0,16.0,...,1.0,0.0,2.0,3.0,3.0,2.0,2.0,10,9.0,10.0
4122,1,1978,2017,1327,1,87.0,0.0,2.0,1.0,4.0,...,1.0,0.0,0.0,3.0,3.0,3.0,2.0,3,7.0,11.0


In [17]:
numeric_data = torch.from_numpy(numeric_values.values[train_indices,1:]).float()
categorical_data = torch.from_numpy(categorical_values.values[train_indices]).float()
targets = torch.from_numpy(numeric_values.values[train_indices,0]).float()

test_numeric_data = torch.from_numpy(numeric_values.values[~train_indices,1:]).float()
test_categorical_data = torch.from_numpy(categorical_values.values[~train_indices]).float()
test_targets = torch.from_numpy(numeric_values.values[~train_indices,0]).float()

In [18]:
train_dataset = data.TensorDataset(numeric_data,categorical_data,targets)
test_dataset = data.TensorDataset(test_numeric_data,test_categorical_data,test_targets)

In [29]:
def get_accuracy(model, data_loader):
    correct = 0
    total = 0
    model.eval() #*********#
    for x, cat_x, labels in data_loader:
        x, cat_x, labels = x.to(device), cat_x.to(device), labels.to(device)
        output = model(x, cat_x)
        pred = torch.round(output)
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += x.shape[0]
    return correct / total

In [30]:
class Flats_classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.emb_layer = nn.Linear(categorical_data.shape[1], categorical_data.shape[1])
        self.act_emb = nn.Tanh()
        self.layer1 = nn.Linear(numeric_data.shape[1] + categorical_data.shape[1], 100)
        self.batch_norm1 = nn.BatchNorm1d(100)
        self.act_1 =  nn.LeakyReLU()
        self.d1 = nn.Dropout(0.4)
        self.layer2 = nn.Linear(100, 50)
        self.batch_norm2 = nn.BatchNorm1d(50)
        self.act_2 =  nn.LeakyReLU()
        self.d2 = nn.Dropout(0.4)
        self.layer3 = nn.Linear(50, 1)
        self.act_out = nn.Sigmoid()
    def forward(self, x, cat_x):
        cat_x_embedded = self.emb_layer(cat_x)
        cat_x_embedded = self.act_emb(cat_x_embedded)
        x = torch.cat([x,cat_x_embedded],dim=1)
        activation1 = self.act_1(self.batch_norm1(self.layer1(x)))
        activation1 = self.d1(activation1)
        activation2 = self.act_2(self.batch_norm2(self.layer2(activation1)))
        activation2 = self.d2(activation2)
        output = self.act_out(self.layer3(activation2))
        return output

In [31]:
model = Flats_classifier().to(device)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=False)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

iters = []
losses = []
train_acc = []
val_acc = []
for n in range(100):
    epoch_losses = []
    for x, cat_x, labels in iter(train_loader):
        x, cat_x, labels = x.to(device), cat_x.to(device), labels.to(device)
        model.train() 
        out = model(x, cat_x).squeeze()           

        loss = criterion(out, labels)
        loss.backward()
        epoch_losses.append(loss.item())
        optimizer.step()
        optimizer.zero_grad()

    loss_mean = np.array(epoch_losses).mean()
    iters.append(n)
    losses.append(loss_mean)
    test_acc = get_accuracy(model, test_loader)
    print(f"Epoch {n} loss {loss_mean:.3} test_acc: {test_acc:.3}")
    train_acc.append(get_accuracy(model, train_loader)) # compute training accuracy 
    val_acc.append(test_acc)  # compute validation accuracy
        

print("Final Training Accuracy: {}".format(train_acc[-1]))
print("Final Validation Accuracy: {}".format(val_acc[-1]))

Epoch 0 loss 0.769 test_acc: 0.745
Epoch 1 loss 0.708 test_acc: 0.73
Epoch 2 loss 0.683 test_acc: 0.793
Epoch 3 loss 0.671 test_acc: 0.532
Epoch 4 loss 0.662 test_acc: 0.281
Epoch 5 loss 0.657 test_acc: 0.272
Epoch 6 loss 0.654 test_acc: 0.206
Epoch 7 loss 0.647 test_acc: 0.222
Epoch 8 loss 0.647 test_acc: 0.28
Epoch 9 loss 0.643 test_acc: 0.211
Epoch 10 loss 0.64 test_acc: 0.397
Epoch 11 loss 0.642 test_acc: 0.21
Epoch 12 loss 0.641 test_acc: 0.776
Epoch 13 loss 0.64 test_acc: 0.567
Epoch 14 loss 0.639 test_acc: 0.671
Epoch 15 loss 0.637 test_acc: 0.814
Epoch 16 loss 0.636 test_acc: 0.875
Epoch 17 loss 0.635 test_acc: 0.551
Epoch 18 loss 0.636 test_acc: 0.704
Epoch 19 loss 0.634 test_acc: 0.46
Epoch 20 loss 0.634 test_acc: 0.823
Epoch 21 loss 0.632 test_acc: 0.802
Epoch 22 loss 0.637 test_acc: 0.853
Epoch 23 loss 0.638 test_acc: 0.521
Epoch 24 loss 0.633 test_acc: 0.798
Epoch 25 loss 0.633 test_acc: 0.857
Epoch 26 loss 0.632 test_acc: 0.61
Epoch 27 loss 0.632 test_acc: 0.604
Epoch 28 